In [64]:
import rosbag
import numpy as np
import gtsam as gt
import numpy_ros
import sensor_msgs
import nav_msgs
import time
from tqdm import tqdm
import yaml
from scipy.spatial.transform import Rotation as R

In [65]:
#Load Config file
with open('../parse_rosbag/config/configFile.yaml', 'r') as file:
    config = yaml.safe_load(file)

tooFar = config['tooFar']
ignoredClouds = config['ignoredClouds']

# covariance for odometry edges
odomCovXX = config['odomCovXX']
odomCovYY = config['odomCovYY']
odomCovTT = config['odomCovTT']

# covariance for loop closure edges
maxCovXX = config['maxCovXX']
maxCovYY = config['maxCovYY']
maxCovTT = config['maxCovTT']

#g2o files
Odomfile = open("odom.g2o","a")
GTfile = open("gt.g2o","a")
myfile = open("myfile.g2o","a")

#intitial transformations to be used to from edges
transform_0 = np.eye(4)
transform_t = np.eye(4)
initialized = False

# slam 
slamX = []
slamY = []
slamTheta = []

In [66]:
bag = rosbag.Bag('../parse_rosbag/data/rosbag/5sidedSquare.bag')
vertexCount = 0
for topic, msg, t in bag.read_messages():
    if topic == '/vertex_odom' or topic == '/edge_odom':
        msg = msg.data.split(' ')
        if msg[0] == 'VERTEX_SE2':
            edge_data = np.array(edge[1:]).astype(np.float)
            currentX, currentY, currentYaw = edge_data[2:5]
            Odomfile.write('VERTEX_SE2 '+ str(vertexCount) + ' '+ str(currentX) + ' ' + \
                           str(currentY) + ' ' + str(currentYaw) + '\n')
            if not initialized:
                transform_0[0:3,3] += np.array([currentX,currentY,0]) # translation
                prevR = R.from_matrix(transform_0[0:3,0:3])
                transform_0[0:3,0:3] = R.from_rotvec(prevR.apply(np.array([0,0,currentYaw]))).as_matrix()
                initialized = True
            else:
                transform_t = np.eye(4)
                transform_t[0:3,3] += np.array([currentX,currentY,0])
                transform_t[0:3,0:3] = R.from_rotvec(np.array([0,0,currentYaw])).as_matrix()
    if topic == '/currentPose':
        roll,pitch,yaw = R.from_quat([msg.pose.pose.orientation.x, msg.pose.pose.orientation.y, msg.pose.pose.orientation.z, msg.pose.pose.orientation.w]).as_euler('xyz')
        myfile.write("VERTEX_SE2 "+str(msg.pose.pose.orientation.x) + " " +str(msg.pose.pose.orientation.y)+" "+str(yaw)+"\n")
        slamX.append(msg.pose.pose.orientation.x)
        slamY.append(msg.pose.pose.orientation.y)
        slamTheta.append(yaw)
        
        if vertexCount>0:
            transform_0t = np.linalg.inv(transform_0)@transform_t
            transform_0 = transform_t
            theta = np.arctan(-transform_0t[0, 1], transform_0t[0, 0])
            myfile.write("EDGE_SE2 "+str(vertexCount-1)+" "+str(vertexCount)+" "\
                         +transform_0t[0,3]+" "+str(transform_0t[1,3])+" "+str(theta)+" "\
                         +str(odomCovXX)+" 0 0 "+str(odomCovYY)+" 0 "+str(odomCovTT)+"\n")
            vertexCount=vertexCount+1
    elif topic == '/odom':
        roll,pitch,yaw = R.from_quat([msg.pose.pose.orientation.x, msg.pose.pose.orientation.y, msg.pose.pose.orientation.z, msg.pose.pose.orientation.w]).as_euler('xyz')
        GTfile.write("VERTEX_SE2 "+str(vertexCount)+" "+str(msg.pose.pose.position.x)+" "+str(msg.pose.pose.position.y)+" "+str(yaw)+"\n")
bag.close()
newEdgeCount = 0

for j in range(ignoredClouds,len(slamX)):
    closestMatch = -1
    closestDist = 1000
    for i in range(ignoredClouds-j):
        dist = (slamX[i]-slamX[j])**2 + (slamY[i]-slamY[j])**2
        if dist < tooFar**2:
            if dist < closestDist:
                closestMatch=i
                closestDist=dist
    if closestDist != -1:
        i=closestMatch
        covXX = maxCovXX*abs(slamX[i]-slamX[j])/tooFar
        covYY = maxCovYY*abs(slamY[i]-slamY[j])/tooFar
        myfile.write("EDGE_SE2 "+str(i)+" "+str(j)+" 0 0 "+str(slamTheta[j]-slamTheta[i])\
                     +" "+str(odomCovXX)+" 0 0 "+str(odomCovYY)+" 0 "+str(odomCovTT)+"\n")
        newEdgeCount=newEdgeCount+1

myfile.close();
GTfile.close();
Odomfile.close();

/tmp/ipykernel_230116/627489354.py:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  edge_data = np.array(edge[1:]).astype(np.float)
